### Embedding inversion

Reconstructing text sequences from their embeddings, a process known as **embedding inversion**, poses significant security and privacy risks. While embeddings are numerical representations designed to capture the semantic meaning of text, they can inadvertently retain enough information to allow attackers to recover the original or similar text, leading to various security vulnerabilities.

**Key Security Issues:**

1.  **Data Leakage and Privacy Violations:** This is the most prominent concern. Embeddings derived from sensitive text data—such as personally identifiable information (PII), financial records, medical notes, confidential business communications, or proprietary research—can be reversed.
    * **Exposure of Sensitive Information:** Attackers can reconstruct parts of the original text, revealing names, addresses, medical diagnoses, trade secrets, or other confidential details. Even if the reconstruction isn't perfect, recovering the "gist" or key entities can be highly damaging.
    * **De-anonymization:** Even if the original data was pseudonymized, the reconstructed text might contain enough contextual information to re-identify individuals or entities.

2.  **Membership Inference Attacks:** While not direct text reconstruction, a related risk is that embeddings can be used to determine if a specific piece of text was part of the data used to train the embedding model or if it's present in a vector database. This can reveal sensitive associations or the presence of particular records. For example, an attacker could check if a specific confidential document or a user with certain attributes is part of a dataset.

3.  **Attribute Inference Attacks:** Attackers might be able to infer sensitive attributes about the individuals or entities mentioned in the original text, even if those attributes are not explicitly stated. For example, by analyzing embeddings, an attacker might infer demographic information, sentiments, or relationships that were not meant to be public.

4.  **Intellectual Property Theft:** If embeddings of copyrighted material or proprietary documents are accessible, attackers could attempt to reconstruct these materials, leading to intellectual property theft. Research has shown that it's possible to reconstruct text with enough accuracy to potentially violate copyright laws if the encoder and embeddings are available.

5.  **Model and System Vulnerabilities:**
    * **Exploiting Retrieval Augmented Generation (RAG) Systems:** Many modern AI systems use RAG, where embeddings of private documents are stored in vector databases to provide context to Large Language Models (LLMs). If these embeddings can be inverted, the supposedly private data becomes exposed.
    * **Unauthorized Access to Vector Databases:** If a vector database containing embeddings of sensitive information is compromised, the ability to reconstruct text from these embeddings becomes a critical threat.

**Real-World Examples and Research Findings Illustrating the Risks:**

While publicly documented instances of malicious actors exploiting these vulnerabilities "in the wild" leading to specific data breaches are often kept confidential, extensive research has demonstrated the feasibility and potential impact of these attacks, serving as critical real-world warnings:

1.  **Reconstruction of Sensitive Phrases:** Researchers have shown that even if exact word-for-word reconstruction is difficult, the semantic meaning and key information can be retrieved. For instance, an embedding for "Our earnings are down" might be inverted to something very similar like "Our income is down," which still leaks the critical information. More advanced techniques have shown success in recovering most exact words, including full names.

2.  **"Text Embeddings Reveal (Almost) As Much As Text" Study:** This influential paper by Cornell University researchers demonstrated that a significant portion of original text can be reconstructed from its dense embeddings. Their "Vec2Text" model was able to recover **92% of 32-token text inputs exactly**. This highlights that short, sensitive pieces of information (like messages, notes, or queries) are highly vulnerable.

3.  **MIMIC-III Medical Dataset Case Study:** As part of the research on text inversion, a case study was conducted using the MIMIC-III dataset, which contains clinical notes. The inversion model was able to recover a high percentage of sensitive data, including:
    * **94% of first names**
    * **95% of last names**
    * **80% of full names**
    This is a stark example of how embeddings from highly sensitive medical data can be used to re-identify individuals and expose private health information.

4.  **Transfer Attacks:** Recent research indicates that attackers might not even need direct access to the original embedding model. "Transfer attacks" are being developed where an adversary can use a surrogate model to mimic the victim model's embeddings and then perform inversion. This broadens the threat landscape, as leaked embeddings alone could be sufficient for an attack.

These research findings, often conducted on datasets and models similar to those used in real-world applications (like BERT, T5, OpenAI embeddings), underscore the tangible security risks. Companies building systems that rely on text embeddings for search, RAG, or other NLP tasks must be aware that embeddings are not inherently secure or one-way hashes. They can, and have been shown to, leak information from the original text, making data protection strategies like encryption of embeddings, access controls, and noise injection crucial.

In [11]:
import torch
from openai import AzureOpenAI
import vec2text
from litellm import embedding
from dotenv import load_dotenv
import vec2text.utils.utils
from tenacity import retry, wait_fixed, stop_after_attempt
import os
import math
from concurrent.futures import ThreadPoolExecutor

load_dotenv()

@retry(wait=wait_fixed(1), stop=stop_after_attempt(10))
def azure_openai_embeddings(text_list, model="text-embedding-ada-002"):
    """
    Azure OpenAI compatible embeddings function
    """
    from openai import AzureOpenAI
    
    # Azure OpenAI setup
    client = AzureOpenAI(
        api_key=os.environ["AZURE_API_KEY"],
        api_version=os.environ.get("AZURE_API_VERSION", "2024-10-21"),
        azure_endpoint=os.environ["AZURE_API_BASE"]
    )
    
    batches = math.ceil(len(text_list) / 128)
    outputs = []

    for i in range(len(text_list)):
        if len(text_list[i]) == 0:
            print(f"warning: set element {i} to a random sequence")
            text_list[i] = "random sequence"

    def process_batch(batch):
        text_list_batch = text_list[batch * 128 : (batch + 1) * 128]
        response = client.embeddings.create(
            input=text_list_batch,
            model=model,
            encoding_format="float"
        )
        return [e.embedding for e in response.data]

    with ThreadPoolExecutor() as executor:
        batch_indices = range(batches)
        results = executor.map(process_batch, batch_indices)

        for result in results:
            outputs.extend(result)

    return outputs

# Apply the patch
vec2text.utils.utils.get_embeddings_openai_vanilla_multithread = azure_openai_embeddings

corrector = vec2text.load_pretrained_corrector("text-embedding-ada-002")

def get_embeddings_openai(text_list, model="text-embedding-ada-002") -> torch.Tensor:
  outputs = []
  response = embedding(
      model="azure/text-embedding-ada-002",
      input=text_list,
      encoding_format="float"
  )
  outputs.extend([e["embedding"] for e in response["data"]])
  return torch.tensor(outputs)


embeddings = get_embeddings_openai([
    "Jack Morris is a PhD student at Cornell Tech in New York City",
    "It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity"
])


vec2text.invert_embeddings(
    embeddings=embeddings.cuda(),
    corrector=corrector,
    num_steps=20,
)

['Jack Morris is a PhD student in tech at Cornell University in New York City',
 'It was the best time of the epoch, it was the worst time of the epoch, it was the best time of the age of wisdom, it was the age of incredulity, it was the age of betrayal']